In [12]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-e4623fa2-6faa-6729-3462-1c2d0e3fccc3)


In [13]:
from google.colab import drive 
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
%cd drive/My Drive/dataset/standford_dog_breed_dataset/images

[Errno 2] No such file or directory: 'drive/My Drive/dataset/standford_dog_breed_dataset/images'
/content/drive/My Drive/dataset/standford_dog_breed_dataset/images


In [15]:
from tensorflow.keras.applications import ResNet50,VGG16,MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.datasets import mnist
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, AveragePooling2D, Flatten, Input, Dropout, BatchNormalization
from imutils import paths
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
import os
import numpy as np
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

In [16]:
# construct the training image generator for data augmentation
aug = ImageDataGenerator(
	rotation_range=20,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest")

In [17]:
baseModel = ResNet50(include_top=False, input_shape=(224, 224, 3))
                      #input_tensor=Input(shape=(224, 224, 3)))


In [18]:
# loop over all layers in the base model and freeze them so they will NOT be updated during the first training process
for layer in baseModel.layers:
	layer.trainable = False

In [19]:
# construct the head of the model that will be placed on top of the
# the base model
headModel = baseModel.output
#headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = GlobalAveragePooling2D()(headModel)
headModel = BatchNormalization()(headModel)
#headModel = Flatten(name="flatten")(headModel)
#headModel = Dense(1024, activation="relu")(headModel)
#headModel = Dropout(0.5)(headModel)
headModel = Dense(120, activation="softmax")(headModel)


# place the head FC model on top of the base model (this will become the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)

In [20]:
INIT_LR = 0.0001
EPOCHS = 100
BS = 256
opt = Adam(learning_rate=INIT_LR)
           #decay_step=1000, decay_rate=0.9)
model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])

In [21]:
model.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
_______________________________________________________________________________________

In [ ]:
H = model.fit(aug.flow_from_directory("Images", batch_size=BS,target_size=(224, 224)), epochs=EPOCHS, verbose=1)

Found 20254 images belonging to 120 classes.
Epoch 1/100
80/80 [==============================] - 7105s 89s/step - loss: 4.8956 - accuracy: 0.0418
Epoch 2/100
80/80 [==============================] - 306s 4s/step - loss: 3.6519 - accuracy: 0.1749
Epoch 3/100
80/80 [==============================] - 305s 4s/step - loss: 2.9414 - accuracy: 0.2930
Epoch 4/100
80/80 [==============================] - 310s 4s/step - loss: 2.5325 - accuracy: 0.3753
Epoch 5/100
80/80 [==============================] - 304s 4s/step - loss: 2.2560 - accuracy: 0.4270
Epoch 6/100
80/80 [==============================] - 302s 4s/step - loss: 2.0732 - accuracy: 0.4687
Epoch 7/100
80/80 [==============================] - 302s 4s/step - loss: 1.9317 - accuracy: 0.5011
Epoch 8/100
80/80 [==============================] - 303s 4s/step - loss: 1.8235 - accuracy: 0.5203
Epoch 9/100
80/80 [==============================] - 299s 4s/step - loss: 1.7346 - accuracy: 0.5446
Epoch 10/100
80/80 [==============================] -